In [21]:
import numpy as np 

training_data = np.loadtxt('features.train')
testing_data = np.loadtxt('features.test')

training_xs = training_data[:,1:3]
training_ys = training_data[:,0]

testing_xs = testing_data[:,1:3]
testing_ys = testing_data[:,0]

def make_binary(ys, digit):
    return np.array([1 if y == digit else -1 for y in ys])

def make_one_v_one(xs, ys, digit_1, digit_2):
    ids = (ys == digit_1) | (ys == digit_2)
    return (xs[ids], ys[ids])


def pseudo_inverse_reg(xs, lamb):
    return np.linalg.inv((xs.T @ xs) + (lamb * np.eye(xs.shape[1]))) @ xs.T

# lowest e_in..
# digits = range(0, 9 + 1)

training_xs = np.column_stack((np.ones((len(training_xs), 1)), training_xs))

testing_xs = np.column_stack((np.ones((len(testing_xs), 1)), testing_xs))

training_xs, training_ys = make_one_v_one(training_xs, training_ys, 1, 5)
testing_xs, testing_ys = make_one_v_one(testing_xs, testing_ys, 1, 5)

training_zs = np.column_stack((training_xs, training_xs[:, 1] * training_xs[:, 2], training_xs[:, 1] ** 2, training_xs[:, 2] ** 2))
testing_zs = np.column_stack((testing_xs, testing_xs[:, 1] * testing_xs[:, 2], testing_xs[:, 1] ** 2, testing_xs[:, 2] ** 2))

training_ys = make_binary(training_ys, 1)
testing_ys = make_binary(testing_ys, 1)

for lamb in (.01, 1):
    weights = pseudo_inverse_reg(training_zs, lamb) @ training_ys

    # E_in
    predicted_ys = np.sign(training_zs @ weights)
    misclass_e_in = len(np.flatnonzero(predicted_ys != training_ys))

    # E_out
    predicted_ys = np.sign(testing_zs @ weights)
    misclass_e_out = len(np.flatnonzero(predicted_ys != testing_ys))

    print(lamb, misclass_e_in, misclass_e_out)

    

0.01 7 12
1 8 11


In [28]:
xs = np.array((
    (1,0),
    (0,1),
    (0,-1),
    (-1,0),
    (0,2),
    (0,-2),
    (-2,0)
))
ys = np.array([-1,-1,-1,1,1,1,1])

import numpy as np
from sklearn import svm

clf = svm.SVC(kernel='poly', C=99999999, degree=2, gamma=1, coef0=1)
clf.fit(xs, ys)
print(len(clf.support_vectors_))

5


In [53]:
# RBF questions
from sklearn import cluster

def runs(gamma, k, n=100, run_count=1000):
    # total_non_separable = 0
    # total_svm_win = 0
    # total_e_in = 0
    # total_e_out = 0
    total_zero_e_in = 0
    for i in range(0,run_count):
        # non_separable, svm_win = run(gamma, k, n)
        # e_in, e_out = run(gamma, k, n)
        # total_e_in += e_in
        # total_e_out += e_out
        # total_non_separable += non_separable
        # total_svm_win += svm_win
        zero_e_in = run(gamma, k, n)
        total_zero_e_in += zero_e_in
    
    # return (total_non_separable / run_count) 
    # return (total_svm_win / run_count)
    # avg_e_in = total_e_in / run_count
    # avg_e_out = total_e_out / run_count
    # return (avg_e_in, avg_e_out)
    return (total_zero_e_in / run_count)

def run(gamma, k, n):
    training_xs = np.random.uniform(1, -1, (n,2))
    training_ys = np.sign(training_xs[:, 1] - training_xs[:, 0] + (.25 * np.sin(np.pi * training_xs[:, 0])))

    testing_xs = np.random.uniform(1, -1, (n,2))
    testing_ys = np.sign(testing_xs[:, 1] - testing_xs[:, 0] + (.25 * np.sin(np.pi * testing_xs[:, 0])))

    # clf = svm.SVC(kernel='rbf', C=1e10, gamma=gamma, coef0=1.0)
    # clf.fit(training_xs, training_ys)
    # e_in = 1 - clf.score(training_xs, training_ys)
    # svm_test_score = clf.score(testing_xs, testing_ys)

    kmeans = cluster.KMeans(n_clusters=k, init='random', n_init=1)
    kmeans.fit(training_xs)
    centers = kmeans.cluster_centers_

    phi = np.zeros((n, k))
    for i in range(n):
        for j in range(k):
            phi[i][j] = np.exp(-gamma * (np.linalg.norm(training_xs[i] - centers[j]) ** 2))

    phi = np.c_[np.ones(n), phi]

    weights = np.linalg.inv(phi.T @ phi) @ phi.T @ training_ys

    # Calculate e_in for pure rbf
    predicted_ys = np.sign(np.dot(phi, weights))
    correct_ys = len(np.flatnonzero(predicted_ys == training_ys))
    e_in = 1 - (correct_ys / n)

    # Calculate E_out for pure rbf
    # phi = np.zeros((n, k))
    # for i in range(n):
    #     for j in range(k):
    #         phi[i][j] = np.exp(-gamma * (np.linalg.norm(testing_xs[i] - centers[j]) ** 2))

    # phi = np.c_[np.ones(n), phi]

    # predicted_ys = np.sign(np.dot(phi,weights))

    # correct_ys = len(np.flatnonzero(predicted_ys == testing_ys))

    # e_out = 1 - (correct_ys / n)

    # svm_win = svm_test_score > regular_test_score

    # return svm_win
    return e_in == 0

print(runs(1.5, 9))    

(0.0334200000000001, 0.058990000000000056)
(0.03690000000000003, 0.06301000000000011)
